In [5]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn


from HDRDN_Mamba import HDRDN_Mamba
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
full_model = HDRDN_Mamba(n_channel =8, out_channel = 4, embed_dim = 60, depths=[4, 4, 4])
full_model = full_model.cuda()

In [7]:
npypath = '/data/asim/ISP/HDR_transformer/data/RAW/raw-2022-0606-2151-4147.npz'
imdata = np.load(npypath)

sht = imdata['sht']
mid = imdata['mid']
lng = imdata['lng']
hdr = imdata['hdr']

crop_size = 128
H, W = hdr.shape[1], hdr.shape[2]
start_h = (H - crop_size) // 2
start_w = (W - crop_size) // 2

sht_crop = sht[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
mid_crop = mid[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
lng_crop = lng[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
hdr_crop = hdr[:, start_h:start_h+crop_size, start_w:start_w+crop_size]

print(sht_crop.shape, hdr_crop.shape)

def to_tensor(np_array):
    t = torch.from_numpy(np_array).float()
    return t

im1 = to_tensor(sht_crop).to(device).unsqueeze(0)
im2 = to_tensor(mid_crop).to(device).unsqueeze(0)
im3 = to_tensor(lng_crop).to(device).unsqueeze(0)
ref_hdr = to_tensor(hdr_crop).to(device).unsqueeze(0)
im1[:, :4, :, :].shape, ref_hdr.shape

(8, 128, 128) (4, 128, 128)


(torch.Size([1, 4, 128, 128]), torch.Size([1, 4, 128, 128]))

In [8]:
with torch.no_grad():
    generate_hdr = full_model(im1, im2, im3)
generate_hdr.shape

torch.Size([1, 16384, 60])


torch.Size([1, 4, 128, 128])